# Explanation of the fibertree union operator

## Introduction

The following cells provide a series of examples of the ```union``` operation on fibers. In general, the ```union``` operation creates a new fiber with an element for each coordinate that exists in either input fiber (i.e., the **union** of their coordinates) and a payload that is a tuple of the corresponding payloads from the input fibers.

First, we include some libraries and provide some dropdown lists to select the display style and type of animation.

In [ ]:
# Begin - startup boilerplate code

import os

def run_prelude(**kwargs):

  switches =  " ".join([ f"--{k}={v}" for (k,v) in kwargs.items()])

  for prelude_file in ['./prelude.py', '../prelude.py']:
    if os.path.exists(prelude_file):
      command=f"{prelude_file} {switches}"
      %run $command
      return
    
  print("Downloading prelude.py")
  ! curl -LJOs https://raw.githubusercontent.com/Fibertree-Project/fibertree-notebooks/colab/notebooks/prelude.py
  command=f"./prelude.py {switches}"
  %run $command

# End - startup boilerplate code

run_prelude(style="tree", animation='movie')


## Fibertree union operator

One can union the contents of two fibers using the ``or`` (|) operator. That operator takes two fibers as operands, and returns a fiber that has a element for each coordinate that appears in **either** input fiber and a payload that consists of a triple (three element tuple). The first element of the triple is a mask (indicating whether the rest of the triple contains non-empty payloads from only-A, only-B or both-A-and-B. The next two elements of the triple contain the corresponding payloads from the two input fibers. If an input fiber doesn't have a particular coordinate the **default** payload value (typically zero) of that fiber is used in the triple. However, if there is no payload at a particular coordinate in either input fiber then that coordinate will not appear in the output - note the absense of coordinate 1 in result for the example below.

In [ ]:
#
# Create two rank-1 tensors
#
a_M = Tensor.fromUncompressed(["M"], [1, 0, 3, 0, 5, 0, 7])
b_M = Tensor.fromUncompressed(["M"], [2, 0, 4, 5])

#
# Get the root fibers of the tensors
#
a_m = a_M.getRoot()
b_m = b_M.getRoot()

#
# Calculate the union of the two fibers
#
z1_m = a_m | b_m

#
# Print the inputs and outputs
#
print("Fiber a_m")
displayTensor(a_m)

print("Fiber b_m")
displayTensor(b_m)

print("Fiber a_m | b_m")
displayTensor(z1_m)
print(f"{z1_m:n*}")

The **default** payload of a the result of ```union``` operation is a tuple of the form of the payloads in the result fiber. This default payload will be used for operations such as an insertion on a new coordinate into the fiber (e.g., using the ```<<``` operator). It also is the shape of the payload returned when the fiber is iterated over. The **default** value for payloads in the fiber ```z1_m``` created above is shown in the cell below, and is an empty string and two scalar zeros.

In [ ]:
#
# Obtain the default payload of the result and print it
#
z1_m_default = z1_m.getDefault()

print(z1_m_default)

## Traversing the result of a union

Traversing the result of a union using a ```for``` loop is like traversing any other fiber, except the payload must match the shape of the **default** payload of the fiber. See below how the payload returned by the interaction is a three element tuple:

In [ ]:
#
# Get the root fibers of the tensors
#
a_m = a_M.getRoot()
b_m = b_M.getRoot()

#
# Traverse the elements of a union operation
#
for c, (mask, a_val, b_val) in a_m | b_m:
    print(f"Coordinate: {c}")
    print(f"Mask: {mask}")
    print(f"A_val: {a_val}")
    print(f"B_val: {b_val}")


## Union of complex payloads

Note that one can take the union of a fiber whose payloads have a more complex type, such as a fiber (e.g., from the top rank of a multirank tensor). This is shown in the example below. Note that neither fiber a or fiber b has a payload at coordinate 3, so the output has no output at that coordinate. Unfortunatly, the image view of a tensor whose payload are tuples with Fibers as an element are sort of messy. So a textual print of the result is also shown.

In [ ]:
#
# Create two rank-2 tensors
#
a_MK = Tensor.fromUncompressed(["M", "K"], [[1, 0, 3, 0, 5, 0, 7],
                                           [2, 2, 0, 3, 0, 0, 8],
                                           [0, 0, 0, 0, 0, 0, 0],
                                           [0, 0, 0, 0, 0, 0, 0],
                                           [4, 0, 5, 0, 8, 0, 9]])

b_MK = Tensor.fromUncompressed(["M", "K"], [[2, 0, 4, 5],
                                            [0, 0, 0, 0],
                                            [3, 4, 6, 0],
                                            [0, 0, 0, 0],
                                            [1, 2, 3, 4]]
                                            )

#
# Get the root fibers of the tensors
#
a_m = a_MK.getRoot()
b_m = b_MK.getRoot()

#
# Calculate the union of the two fibers
#
z2_m = a_m | b_m

#
# Print the inputs and outputs
#
print("Fiber a_m")
displayTensor(a_m)

print("Fiber b_m")
displayTensor(b_m)

print("Fiber a_m | b_m")
displayTensor(z2_m)
print(f"{z2_m:n*}")


The default payload for the above union of fibers with more complex payloads is also more complex. In this case, it is a tuple containing an empty string and two fibers (actually a reference to the constructor for a fiber).

In [ ]:
#
# Obtain the default payload of the result and print it
#
z2_m_default = z2_m.getDefault()

print(z2_m_default)

## Union of asymetric complex payloads

Note that one can take the union of a fibers whose payloads are different types. In this example we union a fiber with fibers as its payloads (e.g., the top rank of a multirank tensor) with a fiber whose payloads are scalars.

In [ ]:
#
# Create another rank-2 tensor (is this the same as above)
#
a_MK = Tensor.fromUncompressed(["M", "K"], [[1, 0, 3, 0, 5, 0, 7],
                                           [0, 0, 0, 0, 0, 0, 0],
                                           [2, 2, 0, 3, 0, 0, 8],
                                           [0, 0, 0, 0, 0, 0, 0],
                                           [4, 0, 5, 0, 8, 0, 9]])
#
# Get the root fibers of the tensors
#
a_m = a_MK.getRoot()
b_m = b_M.getRoot()

#
# Calculate the union of the two fibers
#
z3_m = a_m | b_m

#
# Print the inputs and outputs
#
print("Fiber a_m")
displayTensor(a_m)

print("Fiber b_m")
displayTensor(b_m)

print("Fiber a_m | b_m")
displayTensor(z3_m)
print(f"{z3_m:n*}")


The default payload for the above union of fibers with more complex payloads is also more complex. In this case, it is a tuple containing an empty string and a fiber and a zero.

In [ ]:
#
# Obtain the default payload of the result and print it
#
z3_m_default = z3_m.getDefault()

print(z3_m_default)

## Unions of unions

We can take the union of fiber with a fiber that was already a union of two fibers. This is illustrated in the cell below

Note; That the coordinate 1 of the result has an empty "A" element of the tuple, which was generated from the **default** payload for the result of ```a_m | b_m```. Also there is no coordinate 5 in the result, so **no** input fiber had a non-empty payload for coordinate 5.

In [ ]:
#
# Create another rank-1 tensor
#
c_M = Tensor.fromUncompressed(["M"], [1, 2, 3])

#
# Get the root fibers of the tensors
#
a_m = a_M.getRoot()
b_m = b_M.getRoot()
c_m = c_M.getRoot()

#
# Calculate the union of the three fibers
#
z4_m = (a_m | b_m) | c_m

#
# Print the inputs and outputs
#
print("Fiber a_m")
displayTensor(a_m)

print("Fiber b_m")
displayTensor(b_m)

print("Fiber c_m")
displayTensor(c_m)

print("Fiber (a_m | b_m) | c_m")
displayTensor(z4_m)

# Fiber is too complex to print!
#print(f"{z4_m:n*}")

for m, (mask1, (mask2, a_val, b_val), c_val) in z4_m:
     print(f"mask1: {mask1}")
     print(f"  mask2: {mask2}")
     print(f"    a_val: {a_val}")
     print(f"    b_val: {b_val}")
     print(f"  c_val: {c_val}")



Note the shape of the default payload for the result fiber which contains a nested tuple.

In [ ]:
#
# Obtain the default payload of the result and print it
#
z4_m_default = z4_m.getDefault()

print(z4_m_default)

## Different assocation

Associating the unions differently produces a result that differs in the nesting of the payloads. Note how coordinates 4 and 6 have a payload that contains the  **default** payload for the union of ```b_m``` and ```c_m```.

In [ ]:
#
# Get the root fibers of the tensors
#
a_m = a_M.getRoot()
b_m = b_M.getRoot()
c_m = c_M.getRoot()

#
# Calculate the union of the three fibers
#
z5_m = a_m | ( b_m | c_m )

#
# Print the inputs and outputs
#
print("Fiber a_m")
displayTensor(a_m)

print("Fiber b_m")
displayTensor(b_m)

print("Fiber c_m")
displayTensor(c_m)

print("Fiber a_m | (b_m | c_m)")
displayTensor(z5_m)

# Fiber is too complex to print!
#print(f"{z4_m:n*}")

for m, (mask1, a_val, (mask2, b_val, c_val)) in z5_m:
     print(f"mask1: {mask1}")
     print(f"  a_val: {a_val}")
     print(f"  mask2: {mask2}")
     print(f"    b_val: {b_val}")
     print(f"    c_val: {c_val}")




Note the shape of the default payload for the result fiber, which again is nested tuples, but in this case the third element is a tuple.

In [ ]:
#
# Obtain the default payload of the result and print it
#
z5_m_default = z5_m.getDefault()

print(z5_m_default)

## Union multi-argument operator

To allow a cleaner union of multiple operands the library includes a union operator that takes an arbitrary number of arguments (signature ```Fiber.union(*args)```). The payloads of the result of such a multi-argument union is a payload for each coordinate that exists in **any** of the inpupt arguments and that payload is a tuple comtaining a mask (with letters A-Z) and a entry sourced from the corresponding payload in each input argument fiber (input argument fibers with nothing at that coordinate will use the **default** payload from that fiber). This is illustrated below:

In [ ]:
#
# Get the root fibers of the tensors
#
a_m = a_M.getRoot()
b_m = b_M.getRoot()
c_m = c_M.getRoot()

#
# Calculate the union of the three fibers
#
z6_m = Fiber.union(a_m, b_m, c_m)

#
# Print the inputs and outputs
#
print("Fiber a_m")
displayTensor(a_m)

print("Fiber b_m")
displayTensor(b_m)

print("Fiber c_m")
displayTensor(c_m)

print("Fiber union(a_m, b_m, c_m")
displayTensor(z6_m)

# Fiber is too complex to print!
#print(f"{z4_m:n*}")

for m, (mask1, a_val, b_val, c_val) in z6_m:
     print(f"mask1: {mask1}")
     print(f"  a_val: {a_val}")
     print(f"  b_val: {b_val}")
     print(f"  c_val: {c_val}")


Note the shape of the default payload for the result fiber

In [ ]:
#
# Obtain the default payload of the result and print it
#
z6_m_default = z6_m.getDefault()

print(z6_m_default)

## Multi-argument union with complex payloads

The ```Fiber.union()``` operator also works with more complex payloads

In [ ]:
#
# Create one more rank-2 tensor
#
d_MK = Tensor.fromUncompressed(["M", "K"], [[8, 0, 6, 0],
                                            [0, 0, 0, 0],
                                            [5, 0, 7, 0],
                                            [4, 8, 1, 2],
                                            [1, 2, 3, 4]])

#
# Get the root fibers of the tensors
#
a_m = a_M.getRoot()
b_m = b_M.getRoot()
c_m = c_M.getRoot()
d_m = d_MK.getRoot()

#
# Calculate the union of the three fibers
#
z7_m = Fiber.union(a_m, b_m, c_m, d_m)

#
# Print the inputs and outputs
#
print("Fiber a_m")
displayTensor(a_m)

print("Fiber b_m")
displayTensor(b_m)

print("Fiber c_m")
displayTensor(c_m)

print("Fiber d_m")
displayTensor(d_m)

print("Fiber union(a_m, b_m, c_m, d_m)")
displayTensor(z7_m)

# Fiber is too complex to print!
#print(f"{z4_m:n*}")

for m, (mask1, a_val, b_val, c_val, d_val) in z7_m:
    print(f"mask1: {mask1}")
    print(f"  a_val: {a_val}")
    print(f"  b_val: {b_val}")
    print(f"  c_val: {c_val}")
    print(f"  d_val: {d_val}")
    


Note the shape of the default payload for the resulting fiber from ```Fiber.union()```, where one of the elements of the default payload is a Fiber.

In [ ]:
#
# Obtain the default payload of the result and print it
#
z7_m_default = z7_m.getDefault()

print(z7_m_default)

## Testing area

For running alternative algorithms